In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
import sys
import random
import pyautogui
from PyQt5 import QtWidgets, uic
import time
import csv #엑셀파일
import os
import win32com.client 
from openpyxl import Workbook
import xlwt
import math
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import openpyxl 
import matplotlib.pyplot as plt
import keras
import icon_rc
import webbrowser

Using TensorFlow backend.


In [2]:

import numpy as np

list=np.array([[[0.5877672 , 0.34976745, 0.06246536],
  [0.5890321  ,0.35851398 ,0.05245399],
  [0.58910584 ,0.35983458 ,0.0510596 ],
  [0.58911216 ,0.36001796, 0.05086988]]])
list
list.shape
list[0,3,0]

0.58911216

In [3]:
# LSTM for international airline passengers problem with memory
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [4]:
basic_ui = uic.loadUiType("UI_design.ui")[0]

In [5]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [],[] 
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back):]
        dataX.append(a)
        dataY.append(dataset[i:(i + look_back):])
    
    dataX=np.squeeze(dataX,axis=1) #차원 축소
    dataY=np.squeeze(dataY,axis=1) #차원 축소
    return np.array(dataX), np.array(dataY)

In [6]:
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)

In [7]:
data1=pd.read_excel('x_data.xls') #x좌표 엑셀 데이터 불러옴 
data2=pd.read_excel('y_data.xls') #y좌표 엑셀 데이터 불러옴 
data3=pd.read_excel('speed.xls') #speed좌표 엑셀 데이터 불러옴 

pddata1=pd.DataFrame(data1) 
pddata1.head()
pddata2=pd.DataFrame(data2) 
pddata2.head()
pddata3=pd.DataFrame(data3) 
pddata3.head()

hap=[] #진짜 합
for j in range(len(pddata1)):
    x_data=np.array(pddata1.loc[j])
    y_data=np.array(pddata2.loc[j])
    s_data=np.array(pddata3.loc[j])

    hap1=[] #[x,y,속력] 데이터를 저장하고 있는 리스트
    
    for i in range(len(s_data)): 
        sum=[] #리스트 하나당 임시로 x,y,속력을 저장할 리스트 (for문 돌릴때마다 초기화)
        sum.append(x_data[i]) #x넣음
        sum.append(y_data[i]) #y넣음
        sum.append(s_data[i]) #속력넣음
        hap1.append(sum) #[x,y,속력] 하나의 리스트를 hap리스트에 넣음  

    j = j + 1
    
    hap.append(hap1)

n1 = np.array(hap)
print(n1.shape)

(214, 99, 3)


In [8]:
num_sample = n1.shape[0] # 257개 Sample 데이터
num_sequence = n1.shape[1] # 99개 시계열 데이터
num_feature = n1.shape[2] #3개 Feature

dataset = n1

In [9]:
train_size = int(len(dataset) * 0.70) #학습 데이터 70%
test_size = len(dataset) - train_size #테스트 데이터 30%
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [10]:
look_back = 1 #이전 시간 단계 입력변수
#X는 지금 t 값이고, Y는 그 다음의 t+1 값임 (즉, X=t, Y=t+1)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [11]:
from keras.layers import TimeDistributed
from keras.layers import InputLayer
from keras.layers import Reshape
from keras.layers import *
from keras.models import *
from keras.utils import *

model = Sequential() # Sequeatial Model 
model.add(LSTM(32,return_sequences=True, input_shape=(num_sequence,num_feature)))  
model.add(Dense(3, activation='softmax')) # 3개의 예측
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #정규화 adam

history = model.fit(trainX, trainY, epochs=10, batch_size=20, validation_split=0.2, verbose=1)

Train on 117 samples, validate on 30 samples
Epoch 1/10
117/117 [==============================] - 1s 12ms/step - loss: 1501.7843 - accuracy: 0.0287 - val_loss: 1358.1945 - val_accuracy: 0.9081
Epoch 2/10
117/117 [==============================] - 0s 3ms/step - loss: 1221.7054 - accuracy: 0.9338 - val_loss: 1289.3022 - val_accuracy: 0.9350
Epoch 3/10
117/117 [==============================] - 0s 3ms/step - loss: 1159.1993 - accuracy: 0.9698 - val_loss: 1243.9556 - val_accuracy: 0.9842
Epoch 4/10
117/117 [==============================] - 0s 3ms/step - loss: 1124.0832 - accuracy: 0.9926 - val_loss: 1213.1657 - val_accuracy: 1.0000
Epoch 5/10
117/117 [==============================] - 0s 3ms/step - loss: 1097.6442 - accuracy: 0.9998 - val_loss: 1190.0874 - val_accuracy: 1.0000
Epoch 6/10
117/117 [==============================] - 0s 3ms/step - loss: 1076.9381 - accuracy: 1.0000 - val_loss: 1171.4148 - val_accuracy: 1.0000
Epoch 7/10
117/117 [==============================] - 0s 3ms/step 

In [12]:
print("정확도: ",(model.evaluate(testX, testY)))

63/63 [==============================] - 0s 934us/step
정확도:  [1007.4989837162078, 0.9836459755897522]


In [13]:
#inverse softmax
def inverse_softmax(s, x):                           # s -> result of softmax , x -> value to do softmax
    e_x = np.exp(x)
    return np.log(s) + np.log(np.sum(e_x))

In [14]:
#버튼 출력 윈도우창   
class WindowClass(QMainWindow, basic_ui):
    def __init__(self):
        super().__init__()
        self.setupUi(self)

        self.cnt = 0
        
        self.x_data=[]
        self.y_data=[]
        self.speed=[]
        self.n = 0

        self.excel_btn.clicked.connect(self.Start) #이 버튼을 클릭할 경우,마우스 좌표 0.5초동안 데이터 수집

        self.show()
        
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())
    
    def Start(self):
         
            
            line_x = []
            line_y = []
            line_speed=[]

            for j in range(100):     #1초동안 데이터 수집 
                x,y=pyautogui.position()
                line_x.append(x)
                line_y.append(y)   # 안쪽 리스트에 0 추가
                time.sleep(0.001)

            self.x_data.append(line_x)
            self.y_data.append(line_y)
            
            for i in range(len(line_x)):
                Speed=((line_x[i]-line_x[i-1])**2) + ((line_y[i]-line_y[i-1])**2)
                Speed=math.sqrt(Speed)
                line_speed.append(Speed)
            self.speed.append(line_speed)
            
            testSet=[]
            
            for i in range(1,len(line_x)):
            
                test=[] #리스트 하나당 임시로 x,y,속력 저장 리스트
                test.append(line_x[i])
                test.append(line_y[i])
                test.append(line_speed[i])
                testSet.append(test)
            
            Test=[]
            Test.append(testSet)
            
            Test_set = np.array(Test)
            
            print(Test_set)
            print(Test_set.shape)
            
            #모델에 n1 넣음
            
            testPredict = model.predict(Test_set)
            print(testPredict)                                                #testPredict 0번째 1번째 
            
            test_x=Test_set[0,98,0]
            test_y=Test_set[0,98,1]
            pred_x=testPredict[0,98,0]
            pred_y=testPredict[0,98,1]
            
            resultX= inverse_softmax(pred_x, test_x)
            resultY = inverse_softmax(pred_y, test_y)
            
            print(resultX)
            print(resultY)
            x_position=int(resultX)
            y_position=int(resultY)
            
            
            n=0  #결과에따라 사이트 이동
            while(n==0):
                if x_position>400 and x_position<950 :
                    if 320<y_position and y_position<480:
                        #kwu
                        url = 'https://www.kw.ac.kr/ko/'
                        webbrowser.open(url)
                    if 480<y_position and y_position<800:
                        #naver
                        url = 'https://www.naver.com/'
                        webbrowser.open(url)
                if 950<x_position and x_position<1100:
                    if 320<y_position and y_position<480:
                        #google
                        url = 'https://www.google.co.kr/'
                        webbrowser.open(url)
                    if 480<y_position and y_position<800:
                        #youtube
                        url = 'https://www.youtube.com/'
                        webbrowser.open(url)
                n+=1;
                              

app = QApplication([])
ex = WindowClass()
sys.exit(app.exec_())

[[[ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            0.        ]
  [ 904.          674.            

C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


OverflowError: cannot convert float infinity to integer

SystemExit: 0

C:\Users\USER\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
